<a id="title"></a>
# Clustering using K-Means, GMMs, and HDBSCAN

This notebook assumes you are familiar with basic machine learning vocabulary.

---

## Table of Contents
[Introduction](#intro) <br>
[0. Imports](#imports) <br>
[1. MNIST dataset and scaling](#mnist) <br>
[2. Reduce MNIST using UMAP](#umap) <br>
[3. K-Means Clustering](#kmeans) <br>
- [3a. Fit, predict and visualize using training set](#kmeans_train) <br>
- [3b. Predict test set and visualize boundaries](#kmeans_test) <br>

[4. Gaussian Mixture Models (GMMs)](#gmm) <br>
- [4a. Fit, predict, and visualize using training set](#gmm_train) <br>
- [4b. Predict test set and visualize boundaries](#gmm_test) <br>
- [4c. Probabilistic modeling](#gmm_prob) <br>

[5. Hierarchical Density-Based Spatial Clustering of Applications with Noise (HDBSCAN)](#hdbscan) <br>
- [5a. Fit, predict, and visualize using training set](#hdbscan_train) <br>
- [5b. Predict test set and visualize boundaries](#hdbscan_test) <br>
- [5c. Detecting outliers](#hdbscan_anom) <br>

[6. Conclusions](#con) <br>
[Additional Resources](#add) <br>
[About this Notebook](#about) <br>
[Citations](#cite) <br>

<a id="intro"></a>
## Introduction

Finding insightful and valuable patterns in data is difficult. A common approach to better understand data is to find groups of similar samples, and analyze characteristics internally within a group, and externally between groups. Clustering is a subset of machine learning techniques that groups samples based on similarities, and in some cases, determines the number of groups within a dataset. We can cluster data to better understand its complexity, and the relationships between samples. In addition, labeling data is an expensive task for supervised learning. Clustering can "label" unlabeled data using the data's inherent structure, which can then be used for downstream tasks (e.g. regression, classification, etc.).

**The purpose of this notebook is to demonstrate K-Means Clustering, Gaussian Mixture Models (GMMs), and Hierarchical Density-Based Spatial Clustering of Applications with Noise (HDBSCAN) as clustering techniques on the MNIST dataset.**

<a id="imports"></a>
## 0. Imports

We use `numpy` for arrays, `matplotlib` for plotting, `scipy` for scientific functions, `tensorflow` for loading MNIST, and `umap` for reducing MNIST from a high dimensional space to a low dimensional space. In addition, we `sklearn` for k-means and GMMs, and `hdbscan` for HDBSCAN.

If you do not have some of the packages, please follow the installation guides:
- [Numpy](https://numpy.org/install/)
- [Matplotlib](https://matplotlib.org/stable/users/installing/index.html)
- [Scipy](https://scipy.org/install/)
- Tensorflow ([conda](https://docs.anaconda.com/anaconda/user-guide/tasks/tensorflow/) and [pip](https://www.tensorflow.org/install))
- [UMAP](https://umap-learn.readthedocs.io/en/latest/)
- [Scikit-learn](https://scikit-learn.org/stable/install.html)
- [HDBSCAN](https://github.com/scikit-learn-contrib/hdbscan?tab=readme-ov-file#installing)

In [ ]:
from time import time

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import scipy

import tensorflow as tf
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from hdbscan import HDBSCAN, prediction
from umap import UMAP

<a id="mnist"></a>
## 1. MNIST dataset and scaling

MNIST is a popular image dataset of handwritten digits from 0 to 9. We use it to showcase the different clustering techniques. Here are some qualities of the dataset:
- 60k training samples
- 10k testing samples
- 10 classifications (digits 0-9)
- 28x28 images
- 8-bit gray scaled (0-255 pixel values)

Why is MNIST such a good dataset to use for learning ML? 
- Relatively small images (784 features)
- Relatively large dataset (70k samples)
- 10 unique well defined labels (all the digits are clearly different from each other)
- Very clean dataset (little noise)
    - Backgorund pixels are 0 and signal pixels are nearly 255 so it approximates a binomial distibution
    - The digits are well centered, meaning pixels for similar parts of a digit should consistently be in the same vicinity
    
We retrieve our data using `tensorflow`.

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

We define some global variables and min-max normalize the images so pixels range between 0-1 (normalizing data is a common practice in machine learning). We also flatten our 28x28 images into a 784 feature arrays, in which each pixel is a feature.

In [ ]:
# Global variables
x_train_size = x_train.shape[0]
x_test_size = x_test.shape[0]
x_length = x_train.shape[1]
norm = x_train.max()
rs = 42

# Scale images
x_train_scale = x_train / norm
x_test_scale = x_test / norm

# Flatten arrays
x_train_scale_flat = x_train_scale.reshape(x_train_size, x_length ** 2)
x_test_scale_flat = x_test_scale.reshape(x_test_size, x_length ** 2)

Here are the first 16 samples in the training set.

In [ ]:
fig, axs = plt.subplots(4,4,figsize=[10,10])
for i in range (4):
    for j in range (4):
        axs[i,j].imshow(x_train_scale[i*4+j])
plt.tight_layout()

<a id="umap"></a>
## 2. Reduce MNIST using UMAP

Before clustering MNIST, we use dimensionality reduction to reduce our data from a 784D space to a 2D space. In a high dimensional space, the distance between most samples are too similar (i.e. curse of dimensionality). Since most clustering techniques use a distance metric for determining clusters, this quality of our data is a clear weakness. However, dimensionality reduction embeds similar samples near one another, and disimilar samples away from one another, making the distance metric more viable. Here, we use Uniform Manifold Approximation and Projection (UMAP) as our dimensionality reduction technique for its quality in embedding data, and its fast performance speed. For more information on this technique and dimensionality reduction in general, see `scikit_tutorial_mnist_dimred.ipynb`.

The two main hyperparameters for UMAP are `n_neighbors` and `min_dist`. The former determines the number of neighbors used for the local approximation, and larger values conserve more global structure. The latter determines how tight the data is in the reduced space, and smaller values conserve more local structure. Tuning these hyperparameters smoothly changes the overall embedding. We reduce to two components and choose the default hyperparameter values.

In [ ]:
umap = UMAP(n_components=2, n_neighbors=15, min_dist=0.1, random_state=rs)

We fit and transform the scaled MNIST training data using UMAP.

**Note: This should take no longer than a few minutes on a standard Mac laptop**

In [ ]:
umap_mnist_train = umap.fit_transform(x_train_scale_flat)

We confirm the size of the reduced data.

In [ ]:
umap_mnist_train.shape

We plot the data with labels, where each data point represents one image.

In [ ]:
fig, axs = plt.subplots(1,2,figsize=[20,10])
axs[0].grid()
axs[0].set_title('UMAP MNIST (Train; Unlabeled)')
axs[0].scatter(umap_mnist_train[:, 0], umap_mnist_train[:, 1], s=1, alpha=0.5)
axs[0].set_xlabel('UMAP1')
axs[0].set_ylabel('UMAP2')

axs[1].grid()
axs[1].set_title('UMAP MNIST (Train; Labeled)')
for i in range (10):
    mask = y_train == i
    axs[1].scatter(umap_mnist_train[:, 0][mask], umap_mnist_train[:, 1][mask], 
                   s=1, alpha=0.5, label=i, color='C{}'.format(i))
axs[1].set_xlabel('UMAP1')
axs[1].set_ylabel('UMAP2')
axs[1].legend()

UMAP does an excellent job reducing MNIST to separate digits with little noise. 

We also tranform the test set to UMAP space, check its size, and plot.

In [ ]:
umap_mnist_test = umap.transform(x_test_scale_flat)

In [ ]:
umap_mnist_test.shape

In [ ]:
fig, axs = plt.subplots(1,2,figsize=[20,10])
axs[0].grid()
axs[0].set_title('UMAP MNIST (Test; Unlabeled)')
axs[0].scatter(umap_mnist_test[:, 0], umap_mnist_test[:, 1], s=1, alpha=0.5)
axs[0].set_xlabel('UMAP1')
axs[0].set_ylabel('UMAP2')

axs[1].grid()
axs[1].set_title('UMAP MNIST (Test; Labeled)')
for i in range (10):
    mask = y_test == i
    axs[1].scatter(umap_mnist_test[:, 0][mask], umap_mnist_test[:, 1][mask], 
                   s=1, alpha=0.5, label=i, color='C{}'.format(i))
axs[1].set_xlabel('UMAP1')
axs[1].set_ylabel('UMAP2')
axs[1].legend()

Since the test set samples fall directly on the training set samples, we confirm UMAP has generalized to new data. Now with the UMAP MNIST data, we can cluster as if they were unlabeled.

<a id="kmeans"></a>
## 3. K-Means Clustering

[K-means](https://en.wikipedia.org/wiki/K-means_clustering) is one of the most common centroid-based clustering algorithms. The objective of k-means is to iteratively determine the centroids of k clusters. First, the centroids are placed (or chosen from the data) randomly. Then, the samples that are closest (i.e. using a distance metric) to a centroid are labeled as a part of that cluster. Next, the centroids update their positions to the mean positions of each cluster. The algorithm continues to iterate over relabling samples and updating centroid positions until convergence, or until the total number of iterations is met. K-means also assumes the clusters are similar size and shape.

The algorithm is exceptionally fast and simple, which is why practicioners use this algorithm first for clustering. However, the number of clusters and initial centroid positions heavily influence final results, making it less stable than other algorithms for certain data. Different strategies exist for determining the number of clusters and their initial centroid positions, but that is beyond the scope of this tutorial.

Here are some complementary resources:

- [Stat Quest Video (8 minues, simple language)](https://www.youtube.com/watch?v=4b5d3muPQmA&pp=ygUSayBtZWFucyBjbHVzdGVyaW5n)
- [KMeans: How It Works (8 minutes, intermediate language)](https://www.youtube.com/watch?v=_aWzGGNrcic&pp=ygUSayBtZWFucyBjbHVzdGVyaW5n)
- [Computerphile Video (8 minutes, intermediate lanuage)](https://www.youtube.com/watch?v=yR7k19YBqiw&pp=ygUSayBtZWFucyBjbHVzdGVyaW5n)

<a id="kmeans_train"></a>
### 3a. Fit, predict, and visualize using training set

We use [scikit-learn for KMeans](https://scikit-learn.org/1.5/modules/generated/sklearn.cluster.KMeans.html). The main parameter for k-means is `n_clusters`, which defines the number of clusters. We choose 10 clusters since there are convincingly 10 clusters in the UMAP embedding of MNIST, and we know a priori our data has 10 classifications. By default, `KMeans` also uses the k-means++ algorithm to find equidistant initial centroids.

In [ ]:
kmeans = KMeans(n_clusters=10, random_state=rs)

As a baseline, we fit and predict clusters using the pixels as features.

In [ ]:
kmeans_mnist_train = kmeans.fit_predict(x_train_scale_flat)

Let's check the shape to make sure we have a vector of cluster labels.

In [ ]:
kmeans_mnist_train.shape

Now, we plot the UMAP training set with the k-means cluster labels from pixel features.

In [ ]:
plt.figure(figsize=[10,10])
plt.grid()
plt.title(f'UMAP MNIST (Train; KMeans (Pixels); KMeans++; Iterations={kmeans.n_iter_})')
for i in range (10):
    mask = kmeans_mnist_train == i
    plt.scatter(umap_mnist_train[:, 0][mask], umap_mnist_train[:, 1][mask], 
                s=1, alpha=0.5, label=f'Cluster {i}', color=f'C{i}')
umap_cluster_centers = umap.transform(kmeans.cluster_centers_)
plt.scatter(umap_cluster_centers[:, 0], umap_cluster_centers[:, 1], 
            color='k', label='centroids')
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')
plt.legend()

Using the scaled pixels, the k-means clusters have a lot of overlap, and are not well isolated by class. `KMeans` also took 140 iterations to converge using the scaled pixels.

With our cluster labels, we mean stack the images within each cluster to determine their major characteristics.

In [ ]:
fig, axs = plt.subplots(2,5,figsize=[20,5])
for i in range (2):
    for j in range (5):
        mask = kmeans_mnist_train == (i*5+j)
        axs[i, j].imshow(np.mean(x_train_scale[mask], axis=0))

There's two clusters of 1s, a few clusters for 4/7/9, a few clusters for 3/5/8 and individual clusters for 0/2/6 with lots of noise.

Using a more efficient data representation will yield better results. Thus, we fit and predict clusters using the UMAP MNIST data.

In [ ]:
kmeans = KMeans(n_clusters=10, random_state=rs)

t0_kmeans = time()
kmeans_mnist_train = kmeans.fit_predict(umap_mnist_train)
t1_kmeans = time()

t_kmeans = t1_kmeans - t0_kmeans
print (f'Time spent fitting model: {t_kmeans:.4f} seconds')

K-means fit exceptionally fast. Now, we plot the UMAP training set with the k-means cluster labels from the UMAP embedding.

In [ ]:
plt.figure(figsize=[10,10])
plt.grid()
plt.title(f'UMAP MNIST (Train; KMeans; KMeans++; Iterations={kmeans.n_iter_})')
for i in range (10):
    mask = kmeans_mnist_train == i
    plt.scatter(umap_mnist_train[:, 0][mask], umap_mnist_train[:, 1][mask], 
                s=1, alpha=0.5, label=f'Cluster {i}', color=f'C{i}')
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], 
            color='k', label='centroids')
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')
plt.legend()

KMeans++ clusters our data to a mediocre level. While the purple and gray clusters are well isolated, some clusters are divided too much (pink and green; red and cyan), and others are divided too little (brown and blue; yellow and orange). 

Next, we choose a random set of points as initial centroids by changing the method for initialization (`init`).

In [ ]:
# Fit using random points as initialization
kmeans = KMeans(n_clusters=10, init='random', random_state=rs)
kmeans_mnist_train = kmeans.fit_predict(umap_mnist_train)

# PLot
plt.figure(figsize=[10,10])
plt.grid()
plt.title(f'UMAP MNIST (Train; KMeans; Random; Iterations={kmeans.n_iter_})')
for i in range (10):
    mask = kmeans_mnist_train == i
    plt.scatter(umap_mnist_train[:, 0][mask], umap_mnist_train[:, 1][mask], 
                    s=1, alpha=0.5, label=f'Cluster {i}', color=f'C{i}')
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], 
            color='k', label='centroids')
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')
plt.legend()

With random centroids, nearly six clusters are well isolated (brown, green, purple, cyan, and somewhat red/blue). One cluster is divided too much (gray and pink), and a few others are divided too little (orange and yellow).

We can also choose an array of initial centroids using `init`. Here, we choose the initial centroids as the mean UMAP embedding for each class. In a real world scenario, we may not have labels, but we make use of them to demonstrate this functionality.

In [ ]:
# Fit using custom centroids as initialization
init_centroids = np.array([np.mean(umap_mnist_train[y_train==i], axis=0) for i in range(10)])
kmeans = KMeans(n_clusters=10, init=init_centroids, random_state=rs)
kmeans_mnist_train = kmeans.fit_predict(umap_mnist_train)

# Plot
plt.figure(figsize=[10,10])
plt.grid()
plt.title(f'UMAP MNIST (Train; KMeans; Custom Centroids; Iterations={kmeans.n_iter_})')
for i in range (10):
    mask = kmeans_mnist_train == i
    plt.scatter(umap_mnist_train[:, 0][mask], umap_mnist_train[:, 1][mask], 
                    s=1, alpha=0.5, label=f'Cluster {i}', color=f'C{i}')
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], 
            color='k', label='centroids')
plt.scatter(init_centroids[:, 0], init_centroids[:, 1], 
            facecolors='none', edgecolors='k', label='init')
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')
plt.legend()

With custom centroids, nearly all of our clusters are well isolated, with some overlap in red/brown and gray/purple/cyan. Intuitively, pre-determined centroids from a priori domain knowledge led to the best results. In addition, the initial centroids (open black) did not move far from the fitted centroids, indicating good initialization.

<a id="kmeans_test"></a>
### 3b. Predict test set and visualize boundaries

Now that k-means is trained, we predict the cluster labels on the test set.

In [ ]:
kmeans_mnist_test = kmeans.predict(umap_mnist_test)

We plot the test set with the cluster labels.

In [ ]:
plt.figure(figsize=[10,10])
plt.grid()
plt.title(f'UMAP MNIST (Test; KMeans; Custom Centroids)')
for i in range (10):
    mask = kmeans_mnist_test == i
    plt.scatter(umap_mnist_test[:, 0][mask], umap_mnist_test[:, 1][mask], 
                    s=1, alpha=0.5, label=f'Cluster {i}', color=f'C{i}')
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')
plt.legend()

K-means generalizes to the test set as well.

With our cluster labels, we mean stack the images within each cluster to determine their major characteristics.

In [ ]:
fig, axs = plt.subplots(2,5,figsize=[20,5])
for i in range (2):
    for j in range (5):
        mask = kmeans_mnist_test == (i*5+j)
        axs[i, j].imshow(np.mean(x_test_scale[mask], axis=0))

Since each cluster is well isolated, each mean stack clearly represents a distinct digit.

To visualize the cluster boundaries, we can predict the cluster labels on a grid of points in our manifold. We define a function that creates a grid of points based on our features, plots the cluster labeled data, and plots the clustered labeled grid with high transparency to act as a background.

In [ ]:
def plot_cluster_boundaries(X, Y, model, name, num=200):
    """Plot cluster boundaries.

    Parameters
    ----------
    X : np.array
        Features as a 2D array.
    Y : np.array
        Labels as a 1D array.
    model : sklearn.model
        Clustering model to predict labels.
    name : str
        Name for the plot. Allowed values are 'KMeans', 'GMM', and 'HDBSCAN'.
    num : int, default=200
        Number of x/y coordinates. The number of grid points is num**2.
        
    Returns
    -------
    Z_grid : np.array
        Clutser labels for grid of points as a 2D array (num, num).
    """
    # Make grid points
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xs = np.linspace(x_min, x_max, num)
    ys = np.linspace(y_min, y_max, num)
    x_grid, y_grid = np.meshgrid(xs, ys)
    grid_coords = np.c_[x_grid.ravel(), y_grid.ravel()].astype(np.float32)

    # Predict cluster labels of grid points
    if name in ['KMeans', 'GMM']:
        Z = model.predict(grid_coords)
    elif name == 'HDBSCAN':
        Z, Z_proba = prediction.approximate_predict(model, grid_coords)
        #return Z_proba.reshape(x_grid.shape)
    else:
        raise ValueError(f'`name` is {name}. Use "KMeans", "GMM", or "HDBSCAN".')

    # Plot
    plt.figure(figsize=[10,10])
    plt.title(f'{name} Cluster Boundaries')
    for i in range (Z.max() + 1):
        # Boundaries
        mask_z = Z == i
        plt.scatter(grid_coords[:, 0][mask_z], grid_coords[:, 1][mask_z], 
                    color=f'C{i}', alpha=.025, marker='s')
        # Data points
        mask_xy = Y == i
        plt.scatter(X[:, 0][mask_xy], X[:, 1][mask_xy], 
                    color=f'C{i}', s=1, alpha=0.5, label=f'Cluster {i}')
    # Outliers
    if name == 'HDBSCAN':
        mask_z = Z == -1
        plt.scatter(grid_coords[:, 0][mask_z], grid_coords[:, 1][mask_z], 
                    color='k', alpha=.025, marker='s')
        mask_xy = Y == -1
        plt.scatter(X[:, 0][mask_xy], X[:, 1][mask_xy], 
                    color='k', s=1, alpha=0.5, label=f'Outliers (Hard)')
    plt.xlim(x_min, x_max)
    plt.ylim(y_min, y_max)
    plt.xlabel('UMAP1')
    plt.ylabel('UMAP2')
    plt.legend(loc='lower right')
    plt.show()

    Z_grid = Z.reshape(x_grid.shape)
    return Z_grid

In [ ]:
Z_grid_kmeans = plot_cluster_boundaries(umap_mnist_train, kmeans_mnist_train, kmeans, 'KMeans')

The k-means cluster boundaries are relatively sharp and vary in size. Any point well outside of a cluster takes on the nearest cluster label.

Generally, k-means clustering was limited in its abilities without a priori knowledge. The next algorithm models the probability distribution of the data, making clusters more round and smooth.

<a id="gmm"></a>
## 4. Gaussian Mixture Models (GMMs)

Gaussian mixture models (GMMs), a special case of [mixture models](https://en.wikipedia.org/wiki/Mixture_model), are probabilistic models for distribution-based clustering, and are seen as a generalization of k-means. The algorithm assumes the clusters are generated from a mixture of Gaussians, which gives the clusters well-behaved properties, such as being round and smooth, varying in density, and mitigating class imbalance. A key advantage of GMMs over k-means is incorporating covariance structures in addition to mean structures. Furthermore, GMMs infers the probability of a sample belonging to a cluster (i.e. soft clustering), instead of determining the cluster (i.e. hard clustering). Since it's a generative model, GMMs can also sample new data points after fitting.

GMMs implement [expectation-maximization (EM)](https://en.wikipedia.org/wiki/Expectation%E2%80%93maximization_algorithm), which is an iterative method to maximize likelihood with incomplete data. In our case, the incomplete data are the Gaussians' mean vectors, covariance matrices, and weights. EM first initializes the unknown parameters. Then, it iteratively calculates a posterior distribution as the expected values from the log-likelihood function given current parameter estimates, and updates the parameters to maximize the expected log-likelihood. Once converged, the optimal parameters can be used to predict the cluster of a new sample. However, EM can get stuck in local maxima so fitting a few times and choosing the best model with the maximum likelihood may be necessary. Contrasting to k-means, GMMs scale poorly with the dimensionality of the data so, if possible, fitting on a smaller feature space is optimal.
 
Here are several complementary resources:

- [Six Sigma Pro Smart Video (10 minutes, simple language)](https://www.youtube.com/watch?v=C7jhwN6H9LU)
- [GMMs Explained with Code Video (7 minutes, intermediate language)](https://www.youtube.com/watch?v=hJLaHWaLsyg)
- [GMM Application to Image Segmentation Video (16 minutes, intermediate language)](https://www.youtube.com/watch?v=0nz8JMyFF14)
- [GMM Application to Image Classification Video (12 minutes, intermediate language)](https://www.youtube.com/watch?v=DODphRRL79c)
- [Ritvikmath Video (15 minutes, advanced language)](https://www.youtube.com/watch?v=EWd1xRkyEog)
- [DataMListic Video (5 minutes, advanced language)](https://www.youtube.com/watch?v=wT2yLNUfyoM)
- [GMMs and EM Video (17 minutes, advanced language)](https://www.youtube.com/watch?v=qMTuMa86NzU&list=WL)
- [GMM Tutorial Paper](http://leap.ee.iisc.ac.in/sriram/teaching/MLSP_16/refs/GMM_Tutorial_Reynolds.pdf)

<a id="gmm_train"></a>
### 4a. Fit, predict, and visualize using training set

We use [scikit-learn for GMMs](https://scikit-learn.org/stable/modules/generated/sklearn.mixture.GaussianMixture.html). The main parameter for GMMs is `n_components`, which is the number of clusters, to which we fit 10. By default, `GaussianMixture` fits the full covariance matrix and uses the k-means algorithm for initialization. A secondary parameter called `covariance_type` can change the type of covariance matrix to fit, which can reduce complextiy. Another parameter called `init_params` can change how the initial parameters are found. We leave optimizing these parameters as an exercise for the reader.

In [ ]:
gmm = GaussianMixture(n_components=10, random_state=rs)

Since the pixel dimensions are too high to sufficiently fit the full covariance matrix, we directly fit and predict clusters using the UMAP MNIST data.

In [ ]:
t0_gmm = time()
gmm_mnist_train = gmm.fit_predict(umap_mnist_train)
t1_gmm = time()

t_gmm = t1_gmm - t0_gmm
print (f'Time spent fitting model: {t_gmm:.4f} seconds')

For this 2D data, fitting the GMM is slightly slower than k-means, but still relative in speed. Let's check the shape to make sure we have a vector of cluster labels.

In [ ]:
gmm_mnist_train.shape

Now, we plot the UMAP training set with the GMM cluster labels.

In [ ]:
plt.figure(figsize=[10,10])
plt.grid()
plt.title(f'UMAP MNIST (Train; GMM; KMeans; Iterations={gmm.n_iter_})')
for i in range (10):
    mask = gmm_mnist_train == i
    plt.scatter(umap_mnist_train[:, 0][mask], umap_mnist_train[:, 1][mask], 
                s=1, alpha=0.5, label=f'Cluster {i}', color=f'C{i}')
plt.scatter(gmm.means_[:, 0], gmm.means_[:, 1], 
            color='k', label='centroids')
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')
plt.legend()

The GMM performs similarly to k-means, except the boundaries are a little more round than sharp.

We can also set initial mean vectors, covariance matrices, and weights before fitting. Here, we initialize parameters using the training set labels; again in a real world scenario, labels may be unavailable. The weights are the proportions of each class, the mean vectors are the centroids of each class, and the precision matrices (i.e. inverse of covariance matrices) are the precisions of each class.

In [ ]:
# Initial parameters
weights_init = np.unique(y_train, return_counts=True)[1] / y_train.shape[0]
means_init = np.array([np.mean(umap_mnist_train[y_train==i], axis=0) for i in range(10)])
precisions_init = np.linalg.inv([np.cov(umap_mnist_train[y_train==i].T) for i in range(10)])

# Fit GMM using custom parameters
gmm = GaussianMixture(
    n_components=10, 
    weights_init=weights_init, means_init=means_init, precisions_init=precisions_init,
    random_state=rs
)
gmm_mnist_train = gmm.fit_predict(umap_mnist_train)

# Plot
plt.figure(figsize=[10,10])
plt.grid()
plt.title(f'UMAP MNIST (Train; GMM; Custom Parameters; Iterations={gmm.n_iter_})')
for i in range (10):
    mask = gmm_mnist_train == i
    plt.scatter(umap_mnist_train[:, 0][mask], umap_mnist_train[:, 1][mask], 
                s=1, alpha=0.5, label=f'Cluster {i}', color=f'C{i}')
plt.scatter(gmm.means_[:, 0], gmm.means_[:, 1], 
            color='k', label='centroids')
plt.scatter(means_init[:, 0], means_init[:, 1], 
            facecolor='none', edgecolor='k', label='init')
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')
plt.legend()

The clusters are pretty well isolated, with the exception of some overlap in the purple and cyan clusters.

Again, we can search through random initialization seeds to find an optimal fit. Using a seed found a priori, we fit a GMM with better convergence.

In [ ]:
# Fit using optimal random seed
gmm = GaussianMixture(n_components=10, random_state=40)
gmm_mnist_train = gmm.fit_predict(umap_mnist_train)

# Plot
plt.figure(figsize=[10,10])
plt.grid()
plt.title(f'UMAP MNIST (Train; GMM; KMeans (optimal); Iterations={gmm.n_iter_})')
for i in range (10):
    mask = gmm_mnist_train == i
    plt.scatter(umap_mnist_train[:, 0][mask], umap_mnist_train[:, 1][mask], 
                s=1, alpha=0.5, label=f'Cluster {i}', color=f'C{i}')
plt.scatter(gmm.means_[:, 0], gmm.means_[:, 1], 
            color='k', label='centroids')
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')
plt.legend()

All the clusters are well isolated, and the GMM outperform k-means. A small orange cluster to the right of the gray cluster exists, indicating some overlap in the GMM components.

<a id="gmm_test"></a>
### 4b. Predict test set and visualize boundaries

Now that the GMM is trained, we predict the cluster labels on the test set.

In [ ]:
gmm_mnist_test = gmm.predict(umap_mnist_test)

We plot the test set with the GMM cluster labels, and their mean stacks.

In [ ]:
plt.figure(figsize=[10,10])
plt.grid()
plt.title(f'UMAP MNIST (Test; GMM; KMeans (optimal))')
for i in range (10):
    mask = gmm_mnist_test == i
    plt.scatter(umap_mnist_test[:, 0][mask], umap_mnist_test[:, 1][mask], 
                    s=1, alpha=0.5, label=f'Cluster {i}', color=f'C{i}')
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')
plt.legend()

In [ ]:
fig, axs = plt.subplots(2,5,figsize=[20,5])
for i in range (2):
    for j in range (5):
        mask = gmm_mnist_test == (i*5+j)
        axs[i, j].imshow(np.mean(x_test_scale[mask], axis=0))

The GMM generalizes to the test set as well.

To visualize the boundaries, we can predict the cluster labels on a grid of points in our manifold.

In [ ]:
Z_grid_gmm = plot_cluster_boundaries(umap_mnist_train, gmm_mnist_train, gmm, 'GMM')

The GMM cluster boundaries are more round than k-means, although the orange cluster has some overlap with the gray cluster.

<a id="gmm_prob"></a>
### 4c. Probabilistic modeling

As probabilistic models, GMMs possess some enhanced features over k-means. For example, we can predict the probabilities that a sample belongs to each class.

In [ ]:
gmm_mnist_test_proba = gmm.predict_proba(umap_mnist_test)

The returned array should have the shape (number of samples, number of classes). The sum of each row should be one so the sum of the array should be the number of samples. The max probabilities should also correspond with the cluster label.

In [ ]:
print (f'GMM Test Prediction Probabilities Shape: {gmm_mnist_test_proba.shape}')
print (f'GMM Test Prediction Probabilities Sum: {gmm_mnist_test_proba.sum()}')
print (f'GMM Test Prediction Probabilities Max: {(np.argmax(gmm_mnist_test_proba, axis=1) == gmm_mnist_test).sum()}')

We can also sample from the GMM. Here, we retrieve and plot 10,000 samples from the GMM.

In [ ]:
# Sample from GMM
samples_coords, samples_clusters = gmm.sample(10000)

# Plot
plt.figure(figsize=[10,10])
plt.grid()
plt.title(f'GMM Samples')
plt.scatter(samples_coords[:, 0], samples_coords[:, 1], 
            s=1, alpha=0.5, c=samples_clusters, cmap='tab10')
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')
plt.colorbar()

The sampled clusters are Gaussian. Since our GMMs consist of multiple Gaussian components, we can also sample from a single component of the GMM using the mean vectors and covariance matrices. Here, we sample from the first GMM component, and plot the test data clustered with the first component, the samples generated by the full GMM corresponding to the first component, and the samples generated by only the first component.

In [ ]:
# Sample from a GMM component
ind = 0
mean_vector_component = gmm.means_[ind]
cov_matrix_component = gmm.covariances_[ind]
samples_component = np.random.multivariate_normal(mean_vector_component, cov_matrix_component, 1000)

# Plot
plt.figure(figsize=[10,10])
plt.grid()
plt.title(f'Data vs Samples (Full vs One)')
plt.scatter(umap_mnist_test[:, 0][gmm_mnist_test==ind], umap_mnist_test[:, 1][gmm_mnist_test==ind], 
            s=1, alpha=0.5, label='UMAP MNIST')
plt.scatter(samples_coords[:, 0][samples_clusters==ind], samples_coords[:, 1][samples_clusters==ind], 
            s=1, alpha=0.5, label='GMM')
plt.scatter(samples_component[:, 0], samples_component[:, 1], 
            s=1, alpha=0.5, label='Single Component')
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')
plt.legend()

The samples from both processes have similar distributions, and the UMAP MNIST data is constrained within the distributions.

In addition, we can calculate the negative log-likelihood (i.e. NLL) of samples using the GMM. Samples that are more likely to be observed have a smaller NLL. Since the log-likelihood is strictly negative, calculating the NLL is better for visualization. Again, since the GMM consists of multiple Gaussians, we can also calculate the NLL of samples from a single component of the GMM. We define a function to plot contour lines showing the NLL of samples across the UMAP manifold.

In [ ]:
def plot_gmm_nll_contour(X, Y, model, name, num=200, method='multi'):
    """Plot NLL contour lines from the GMM.

    Parameters
    ----------
    X : np.array
        Features as a 2D array.
    Y : np.array
        Labels as a 1D array.
    model : sklearn.model
        Clustering model to calculate NLL.
    name : str
        Name for the plot.
    num : int, default=200
        Number of x/y coordinates. The number of grid points is num**2.
    method : str, default='multi'
        How to calculate NLL. 'multi' uses the full GMM. 'single' uses each component of the GMM separately.
        
    Returns
    -------
    Z_grid_nll : np.array
        NLL for grid of points as a 2D array (num, num).
    """
    # Make grid points
    length = Y.max() + 1
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xs = np.linspace(x_min, x_max, num)
    ys = np.linspace(y_min, y_max, num)
    x_grid, y_grid = np.meshgrid(xs, ys)
    grid_coords = np.c_[x_grid.ravel(), y_grid.ravel()]

    # Plot data points
    plt.figure(figsize=[12,10])
    plt.grid()
    plt.title(f'{name} NLL Contours ({method})')
    plt.xlim(x_min, x_max)
    plt.ylim(y_min, y_max)
    plt.xlabel('UMAP1')
    plt.ylabel('UMAP2')
    for i in range (length):
        mask_xy = Y == i
        plt.scatter(X[:, 0][mask_xy], X[:, 1][mask_xy], 
                    s=1, alpha=0.5, label=f'Cluster {i}')

    # Calculate NLL and plot contours
    if method == 'multi':
        Z_grid_nll = -model.score_samples(grid_coords).reshape(x_grid.shape)
        plt.contour(x_grid, y_grid, Z_grid_nll, 
                    norm=LogNorm(vmin=1.0, vmax=100.0), levels=np.logspace(0, 2, 10))
    elif method == 'single':
        Z_grid_nll = np.zeros((length, num, num))
        for i in range (length):
            mean_vector_component = model.means_[i]
            cov_matrix_component = model.covariances_[i]
            log_weight_component = np.log(model.weights_[i])
        
            Z_grid_nll_component = -(
                log_weight_component + scipy.stats.multivariate_normal.logpdf(
                    grid_coords, mean_vector_component, cov_matrix_component
                ).reshape(x_grid.shape)
            )
            plt.contour(x_grid, y_grid, Z_grid_nll_component, 
                        norm=LogNorm(vmin=1.0, vmax=10.0), levels=np.logspace(0, 1, 10))
            Z_grid_nll[i] = Z_grid_nll_component
    else:
        raise ValueError(f'`method` is {method}. Use "multi" or "single".')
        
    plt.colorbar() 
    plt.legend(loc='lower right')
    plt.show()

    return Z_grid_nll

In [ ]:
Z_grid_nll_multi = plot_gmm_nll_contour(umap_mnist_test, gmm_mnist_test, gmm, 'GMM', method='multi')

The darker contour lines represent more likely regions, and brighter contour lines represent less likely regions. The NLL produced from the GMM is smooth with intuitive local maxima, such as in the middle of the blue/purple/pink clusters, and in the middle of pink/green/yellow clusters.

We also plot contour lines showing the NLL of samples across the manifold for each individual component.

In [ ]:
Z_grid_nll_single = plot_gmm_nll_contour(umap_mnist_test, gmm_mnist_test, gmm, 'GMM', method='single')

The GMM components overlap each other. The samples are most likely near the cluster's center, and least likely near the cluster's edges.

By converting the NLLs of each component to likelihoods, summing along each component, and converting back to NLLs, we retrieve the full GMM NLLs.

In [ ]:
Z_grid_nll_single_sum = -np.log(np.exp(-Z_grid_nll_single).sum(0))
Z_grid_nll_mse  = np.mean((Z_grid_nll_multi - Z_grid_nll_single_sum)**2)
print (f'Mean Squared Error between NLL grids: {Z_grid_nll_mse}')

Similarly, we can calculate the Mahalanobis distance (MD), which is the multivariate generalization of the square of the standard score. MD represents "how much sigma" a data point is away from a distribution. MD is also directly related to the quadratic term of the NLL for multivariate Gaussians. By taking a weighted sum between the data's MDs and the data's probabilities of belonging to a cluster, we derive a "weighted MD" for the full GMM. This weighted MD represents how much sigma a data point is away from the entire distribution. We define a function to plot contour lines showing the MD of samples across the UMAP manifold.

In [ ]:
def plot_gmm_dist_contour(X, Y, model, name, num=200, method='multi'):
    """Plot MD contour lines from the GMM.

    Parameters
    ----------
    X : np.array
        Features as a 2D array.
    Y : np.array
        Labels as a 1D array.
    model : sklearn.model
        Clustering model to calculate MD.
    name : str
        Name for the plot.
    num : int, default=200
        Number of x/y coordinates. The number of grid points is num**2.
    method : str, default='multi'
        How to calculate MD. 'multi' uses the full GMM. 'single' uses each component of the GMM separately.
        
    Returns
    -------
    Z_grid_md : np.array
        MD for grid of points as a 2D array (num, num).
    """
    # Make grid points
    length = Y.max() + 1
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xs = np.linspace(x_min, x_max, num)
    ys = np.linspace(y_min, y_max, num)
    x_grid, y_grid = np.meshgrid(xs, ys)
    grid_coords = np.c_[x_grid.ravel(), y_grid.ravel()]

    # Plot
    plt.figure(figsize=[12,10])
    plt.grid()
    plt.title(f'{name} MD Contours ({method})')
    plt.xlim(x_min, x_max)
    plt.ylim(y_min, y_max)
    plt.xlabel('UMAP1')
    plt.ylabel('UMAP2')

    Z_grid_md = np.zeros((length, num, num))
    for i in range (length):
        # Plot data points
        mask_xy = Y == i
        plt.scatter(X[:, 0][mask_xy], X[:, 1][mask_xy], 
                    s=1, alpha=0.5, label=f'Cluster {i}')
        # Calculate MD for each component
        mean_vector_component = model.means_[i]
        prec_matrix_component = model.precisions_[i]
        Z_grid_md_component = []
        for grid_coord in grid_coords:
            md = scipy.spatial.distance.mahalanobis(grid_coord, mean_vector_component, prec_matrix_component)
            Z_grid_md_component.append(md)
        Z_grid_md_component = np.array(Z_grid_md_component).reshape(x_grid.shape)
        Z_grid_md[i] = Z_grid_md_component

    if method == 'multi':
        # Calculate weighted MD and plot contours
        Z_grid_proba = np.transpose(model.predict_proba(grid_coords).reshape(num, num, length), (2,0,1))
        Z_grid_md = (Z_grid_md * Z_grid_proba).sum(0)
        plt.contour(x_grid, y_grid, Z_grid_md, levels=np.linspace(0,3,4))
    elif method == 'single':
        # Plot MD contours
        for Z_grid_md_component in Z_grid_md:
            plt.contour(x_grid, y_grid, Z_grid_md_component, levels=np.linspace(0,3,4))
    else:
        raise ValueError(f'`method` is {method}. Use "multi" or "single".')
        
    plt.colorbar()
    plt.legend(loc='lower right')
    plt.show()

    return Z_grid_md

In [ ]:
Z_grid_md_multi = plot_gmm_dist_contour(umap_mnist_test, gmm_mnist_test, gmm, 'GMM', method='multi')

From darkest to lightest, the contour lines represent data that are one (blue), two (green), and three (yellow) sigma away from the full GMM. A handful of images are three sigma away from the GMM.

We also plot contour lines showing the MD of samples across the manifold for each individual component.

In [ ]:
Z_grid_md_single = plot_gmm_dist_contour(umap_mnist_test, gmm_mnist_test, gmm, 'GMM', method='single')

The three sigma boundaries overlap for the brown/gray/orange clusters, and the purple/cyan/green clusters. However, the rest of the clusters are more than three sigma away from each other. By applying various thresholds, we can use either the NLL or MD to find anomalies.

Generally, GMMs outperform k-means and have a few probabilistic advantages, such as predicting cluster probabilities, sampling, and decomposing the GMM to individual components. However, a shared weakness between each algorithm is that the number of clusters before fitting is needed. The last algorithm automatically determines the number of clusters in the dataset, and detects outliers. 

<a id="hdbscan"></a>
## 5. Hierarchical Density-Based Spatial Clustering of Applications with Noise (HDBSCAN)

Density-Based Spatial Clustering of Applications with Noise [(DBSCAN)](https://en.wikipedia.org/wiki/DBSCAN) is a density-based clustering algorithm that labels high-density regions as clusters, and low-density regions as outliers. DBSCAN has two main parameters: the maximum distance to be considered a sample's neighbor (i.e. epsilon), and the minimum number of neighbors to be considered a cluster (i.e. minPts). First, the algorithm finds core points, which are samples that have at least the minimum number of neighbors within distance epsilon. If the core point is within epsilon of an existing cluster, then that core point joins the cluster. If the core point is not within epsilon of an existing cluster, then a new cluster is created. Next, the algorithm recursively finds boundary, or reachable, points, which are samples that neighbor a core point, but do not have enough neighbors to be considered core points. Then, the algorithm finds outliers, which are samples that are not neighbors to any core points, and do not have enough neighbors to be considered core points. 

DBSCAN determines the number of clusters during fitting instead of before fitting, and separates outliers from clusters, which are key advantages from the previous algorithms. Since DBSCAN clusters samples by grouping core points and their neighbors, it also clusters arbitrary shapes.

Some disadvantages of DBSCAN over the previous clustering algorithms are determining a new sample's cluster after fitting. Since adding a new sample may change the data's structure, cluster predictions are difficult to infer. DBSCAN also weakly clusters data with a large density variation since constant epsilon and minPts assume all clusters have similar densities. In addition, epsilon does [not scale well with dimesnionality](https://en.wikipedia.org/wiki/Curse_of_dimensionality) since the distances in high dimensions tend to be more similar. 

However, [HDBSCAN](https://hdbscan.readthedocs.io/en/latest/how_hdbscan_works.html), an extension of DBSCAN that also uses hierarchical clustering, alleviates these disadvantages. HDBSCAN creates a cluster tree by "fitting" over various epsilons, and selects the most stable clusters from the tree.
 
Here are several complementary resources:

- [StatQuest DBSCAN Video (9 minutes, simple language)](https://www.youtube.com/watch?v=RDZUdRSDOok)
- [DBSCAN Explained Video (8 minutes, simple language)](https://www.youtube.com/watch?v=4AW_5nYQkuc)
- [DBSCAN Walkthrough Video (11 minutes, intermediate language)](https://www.youtube.com/watch?v=-p354tQsKrs)
- [HDBSCAN SciPy Video (23 minutes, advanced language)](https://www.youtube.com/watch?v=AgPQ76RIi6A)
- [HDBSCAN PyData Video (34 minutes, advanced language)](https://www.youtube.com/watch?v=dGsxd67IFiU)
- [DBSCAN Original Paper](https://cdn.aaai.org/KDD/1996/KDD96-037.pdf)
- [Accelerated HDBSCAN Paper](https://arxiv.org/pdf/1705.07321)

<a id="hdbscan_train"></a>
### 5a. Fit, transform, and visualize using training set

We use a `scikit-learn-contrib` library for [HDBSCAN](https://hdbscan.readthedocs.io/en/latest/). A [scikit-learn implementation of HDBSCAN](https://scikit-learn.org/1.5/modules/generated/sklearn.cluster.HDBSCAN.html) adpated from the former library is also available, but the former has more features and is better optimized so we use that instead. When using the scikit-learn implementation, be sure to check the notes on their documentation for its slight difference in parameter implementation (see this [GitHub Issue](https://github.com/scikit-learn/scikit-learn/issues/27829) for more information).

The main parameter for HDBSCAN is `min_cluster_size`, which is the minimumn number of samples needed for a group to be considered a cluster. We choose the minimum cluster size to be 20. As with other models, we recommend tuning the parameters to find the best fit.

In [ ]:
hdbscan = HDBSCAN(min_cluster_size=20)

Since the pixel dimensions are too high to sufficiently fit, we directly fit and predict clusters using the UMAP MNIST data.

In [ ]:
t0_hdbscan = time()
hdbscan_mnist_train = hdbscan.fit_predict(x_train_scale_flat)
t1_hdbscan = time()

t_hdbscan = t1_hdbscan - t0_hdbscan
print (f'Time spent fitting model: {t_hdbscan:.4f} seconds')

For this 2D data, fitting HDBSCAN is slightly slower than the other models, but still relative in speed. Let's check the shape to make sure we have a vector of cluster labels.

In [ ]:
hdbscan_mnist_train.shape

Let's also check how many distinct clusters were found.

In [ ]:
set(hdbscan_mnist_train)

10 clusters were found (as denoted from 0-9), and outliers were also found (as denoted as -1).

Now, we plot the UMAP training set with the HDBSCAN cluster labels.

In [ ]:
plt.figure(figsize=[10,10])
plt.grid()
plt.title(f'UMAP MNIST (Train; HDBSCAN)')
for i in range (10):
    mask = hdbscan_mnist_train == i
    plt.scatter(umap_mnist_train[:, 0][mask], umap_mnist_train[:, 1][mask], 
                s=1, alpha=0.5, label=f'Cluster {i}', color=f'C{i}')
mask_out = hdbscan_mnist_train == -1
plt.scatter(umap_mnist_train[:, 0][mask_out], umap_mnist_train[:, 1][mask_out], 
            s=1, alpha=0.5, label=f'Outliers (Hard)', color=f'k')
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')
plt.legend()

Several clusters are relatively isolated, however HDBSCAN fails to separate the brown cluster. Two small clusters were identified: blue (to the right of cyan), and yellow (between cyan and gray). In addition, all the outliers (black) surrounded the cyan cluster since those regions were less dense.

As previously mentioned, we can search through the parameter space to find an optimal fit. Using a tuned parameter, we fit HDBSCAN with better convergence. Note we also set a new parameter `prediction_data=True` to predict new data, which will be further explained in the next subsection.

In [ ]:
# Fit using optimal parameter
hdbscan = HDBSCAN(min_cluster_size=31, prediction_data=True)
hdbscan_mnist_train = hdbscan.fit_predict(umap_mnist_train)
weighted_cluster_centroids = np.array([hdbscan.weighted_cluster_centroid(i) for i in range(10)])

# Plot
plt.figure(figsize=[10,10])
plt.grid()
plt.title(f'UMAP MNIST (Train; HDBSCAN (optimal))')
for i in range (10):
    mask = hdbscan_mnist_train == i
    plt.scatter(umap_mnist_train[:, 0][mask], umap_mnist_train[:, 1][mask], 
                s=1, alpha=0.5, label=f'Cluster {i}', color=f'C{i}')
mask_out = hdbscan_mnist_train == -1
plt.scatter(umap_mnist_train[:, 0][mask_out], umap_mnist_train[:, 1][mask_out], 
            s=1, alpha=0.5, label=f'Outliers (Hard)', color=f'k')
plt.scatter(weighted_cluster_centroids[:, 0], weighted_cluster_centroids[:, 1], 
            color='k', label='centroids')
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')
plt.legend()

All the clusters are well isolated, and HDBSCAN performs slightly better than GMMs. In addition, some outliers are detected, and the small clusters from the previous fit are also detected as outliers.

We can visualize the cluster hierarchy by plotting the `condensed_tree_` attribute as a dendrogram. `HDBSCAN` includes other advanced visualizations such as the minimum spanning tree, the single linkage tree, and cluster branches, but MNIST is too large for rendering so exploring these are left as an exercise for the reader.

In [ ]:
selection_palette = [f'C{i}' for i in range(10)]
plt.figure(figsize=[10,10])
hdbscan.condensed_tree_.plot(select_clusters=True, selection_palette=selection_palette, log_size=True)
plt.yscale('log')

The log-scaled y axis is the lambda value, which refers to the "excess of mass", and is inversely proportional to density. Small lambda (top of plot) represents dense regions, and large lambda (bottom of plot) represents sparse regions. As you move down the plot, larger clusters "branch off" into smaller clusters. The leaves are the end of the clusters' lives, and break off into clusters smaller than `min_cluster_size`. The color bar represents the number of data points of each branch in natural log scale. Our found clusters are circled with their respective colors. The length of each branch represents persistence, which is how long a cluster survives before splitting, and indicates a cluster's stability. Large persistence represents more stability, and small persistence represents less stability. All of our clusters are relatively stable. We retrieve each cluster's persistence with the `cluster_persistence_` attribute.

In [ ]:
for i, per in enumerate(hdbscan.cluster_persistence_):
    print (f'Cluster {i}: {per:.3f}')

Cluster 1 (orange) has the largest persistence, and cluster 8 (yellow) has the smallest persistence. These values follow expectations because the orange cluster is well isolated from others, and the yellow cluster is between the brown and cyan clusters.

HDBSCAN also assigns the probability of belonging to the predicted cluster for each sample. Outliers have a probability of 0, and the max probability is 1. We retrieve these probabilities and plot the distribution.

In [ ]:
hdbscan_mnist_train_proba = hdbscan.probabilities_
plt.title('HDBSCAN Probability Histogram')
plt.grid()
plt.hist(hdbscan_mnist_train_proba, bins=100)
plt.xlabel('Probability')
plt.ylabel('Frequency')
plt.yscale('log')

Most samples have a probability of 1 for belonging to their cluster, but there is a small percentage of samples with probabilities between 0.2 and 1.

<a id="hdbscan_test"></a>
### 5b. Predict test set and visualize boundaries

HDBSCAN can cluster a new sample by determining where the sample lies on the fixed cluster tree. In the previous section, we set `prediction_date=True`, which enables predicting labels. If set to `False`, this functionality is turned off. This parameter also enables advanced soft clustering (i.e. probabilities of a sample belonging to each cluster) similar to GMMs, but the implementation is still in the early stages so we leave exploring that as an exercise for the reader. Here, we predict the cluster labels and probabilities on the test set.

In [ ]:
hdbscan_mnist_test, hdbscan_mnist_test_proba = prediction.approximate_predict(hdbscan, umap_mnist_test)

We plot the test set with the HDBSCAN cluster labels, their mean stacks, and their probability distribution.

In [ ]:
plt.figure(figsize=[10,10])
plt.grid()
plt.title(f'UMAP MNIST (Test; HDBSCAN (optimal))')
for i in range (10):
    mask = hdbscan_mnist_test == i
    plt.scatter(umap_mnist_test[:, 0][mask], umap_mnist_test[:, 1][mask], 
                    s=1, alpha=0.5, label=f'Cluster {i}', color=f'C{i}')
mask_out = hdbscan_mnist_test == -1
plt.scatter(umap_mnist_test[:, 0][mask_out], umap_mnist_test[:, 1][mask_out], 
            s=1, alpha=0.5, label=f'Outliers (Hard)', color=f'k')
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')
plt.legend()

In [ ]:
fig, axs = plt.subplots(2,5,figsize=[20,5])
for i in range (2):
    for j in range (5):
        mask = hdbscan_mnist_test == (i*5+j)
        axs[i, j].imshow(np.mean(x_test_scale[mask], axis=0))

In [ ]:
plt.title('HDBSCAN Probability Histogram')
plt.grid()
plt.hist(hdbscan_mnist_train_proba, bins=100, alpha=0.5, label='train')
plt.hist(hdbscan_mnist_test_proba, bins=100, alpha=0.5, label='test')
plt.xlabel('Probability')
plt.ylabel('Frequency')
plt.yscale('log')
plt.legend()

HDBSCAN generalizes to the test set as well due to similar embedding cluster predictions, mean stacks, and probability distributions.

To visualize the boundaries, we can predict the cluster labels on a grid of points in our manifold.

In [ ]:
Z_grid_hdbscan = plot_cluster_boundaries(umap_mnist_train, hdbscan_mnist_train, hdbscan, 'HDBSCAN')

The HDBSCAN cluster boundaries are proportional to the clusters persistence. The green, red, orange, and blue clusters have larger boundaries than the other clusters. Samples that are distant from the clusters are labeled as outliers.

<a id="hdbscan_anom"></a>
### 5c. Detecting outliers

HDBSCAN hard detects global outliers with the label of -1 (i.e. black data points from previous plots). We count the number of outliers in the test set.

In [ ]:
mask_out = hdbscan_mnist_test == -1
print (f'Number of outliers in test set: {mask_out.sum()}')

About 1% of the test samples were found to be outliers. Now, we plot some of their respective images.

In [ ]:
fig, axs = plt.subplots(4,4,figsize=[10,10])
for i in range (4):
    for j in range (4):
        axs[i,j].imshow(x_test_scale[mask_out][i*4+j])
plt.tight_layout()

These outliers are relatively "out of distribution" compared to their parent digits.

HDBSCAN also soft detects local outliers using the GLOSH algorithm, which assigns each sample an outlier score. This score ranges from 0 (least anomalous) to 1 (most anomalous). This algorithm automatically executes when fitting HDBSCAN. We plot the distribution of outlier scores from the train and test sets.

In [ ]:
outlier_scores_train = hdbscan.outlier_scores_
outlier_scores_test = prediction.approximate_predict_scores(hdbscan, umap_mnist_test)
outlier_scores_1 = np.percentile(outlier_scores_train, 99)
plt.title('HDBSCAN Outlier Score Histogram')
plt.grid()
plt.hist(outlier_scores_train, bins=100, alpha=0.5, label='train')
plt.hist(outlier_scores_test, bins=100, alpha=0.5, label='test')
plt.vlines(outlier_scores_1, 0, 10**4, label='train 99th Percentile', color='k')
plt.xlabel('Outlier Scores')
plt.ylabel('Frequency')
plt.yscale('log')
plt.legend()

The outlier scores are exponentially distributed with most samples having a low outlier score, and a few having a large outlier score. Some outliers in the test set have a negative outlier score, which is a limitation of GLOSH on new data. These samples are typical in nature. 

We plot samples with the highest outlier scores.

In [ ]:
outlier_scores_test_inds_16 = np.argsort(outlier_scores_test)[-16:]
fig, axs = plt.subplots(4,4,figsize=[10,10])
for i in range (4):
    for j in range (4):
        ind = outlier_scores_test_inds_16[i*4+j]
        axs[i,j].set_title(f'Outlier Score: {outlier_scores_test[ind]:.3f}')
        axs[i,j].imshow(x_test_scale[ind])
plt.tight_layout()

These outliers are also relatively "out of distribution" compared to their parent digits.

We plot the UMAP MNIST data only showing the top 1% of outliers. Note we are not plotting the hard detected outliers for simplicity.

In [ ]:
plt.figure(figsize=[10,10])
plt.grid()
plt.title(f'UMAP MNIST (Test; HDBSCAN (optimal))')
for i in range (10):
    mask = hdbscan_mnist_test == i
    plt.scatter(umap_mnist_test[:, 0][mask], umap_mnist_test[:, 1][mask], 
                s=1, alpha=0.5, label=f'Cluster {i}', color=f'C{i}')
mask_out_1 = outlier_scores_test > outlier_scores_1
plt.scatter(umap_mnist_test[:, 0][mask_out_1], umap_mnist_test[:, 1][mask_out_1], 
            s=1, alpha=0.5, label=f'Outliers (Soft; Top 1%)', color=f'k')
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')
plt.legend()

Most samples with high outlier scores are intuitively around cluster borders or between clusters.

<a id="con"></a>
## 6. Conclusions

Clustering is an excellent tool for exploring data and understanding its structure. K-means is a fast centroid-based algorithm that clusters data with respect to centroids. Gaussian mixture model (GMM) is a probabilistic distribution-based algorithm that clusters data with respect to Gaussian distributions. Hierarchical Density-Based Spatial Clustering of Applications with Noise (HDBSCAN) is a hierarchical density-based algorithm that clusters data with respect to cluster trees formed from density regions. There are many clustering algorithms, but using these three in unison is a substantial start to any machine learning based exploratory data analysis.

**Thank you and congratulations for completing the notebook!**

In [ ]:
# Time spent for each model
print (f'Time spent fitting KMeans: {t_kmeans:.4f} seconds')
print (f'Time spent fitting GMM: {t_gmm:.4f} seconds')
print (f'Time spent fitting HDBSCAN: {t_hdbscan:.4f} seconds')

In [ ]:
# Cluster Boundaries
fig, axs = plt.subplots(1,3,figsize=[15,5])
axs[0].set_title('KMeans Cluster Boundary')
axs[0].imshow(Z_grid_kmeans, origin='lower')
axs[1].set_title('GMM Cluster Boundary')
axs[1].imshow(Z_grid_gmm, origin='lower')
axs[2].set_title('HDBSCAN Cluster Boundary')
axs[2].imshow(Z_grid_hdbscan, origin='lower')

In [ ]:
# GMM NLL/MD Multi
fig, axs = plt.subplots(1,2,figsize=[10,5])
axs[0].set_title('GMM NLL Multi')
axs[0].imshow(Z_grid_nll_multi, origin='lower', vmin=0, vmax=10)
axs[1].set_title('GMM MD Multi')
axs[1].imshow(Z_grid_md_multi, origin='lower', vmin=0, vmax=3)

In [ ]:
# GMM NLL/MD Single
fig, axs = plt.subplots(2,10,figsize=[30,6])
for i in range (10):
    axs[0, i].set_title(f'GMM NLL Single: Cluster {i}')
    axs[0, i].imshow(Z_grid_nll_single[i], origin='lower', vmin=0, vmax=10)
    axs[1, i].set_title(f'GMM MD Single: Cluster {i}')
    axs[1, i].imshow(Z_grid_md_single[i], origin='lower', vmin=0, vmax=3)
plt.tight_layout()

<a id="add"></a>
## Additional Resources

Machine learning is a dense and rapidly evolving field of study. Becoming an expert takes years of practice and patience, but hopefully this notebook brought you closer in that direction. Here are some of the author's favorite resources for learning about machine learning and data science:

- [Google Machine Learning Crash Course](https://developers.google.com/machine-learning/crash-course)
- [scikit-learn Python Library](https://scikit-learn.org/stable/index.html) (go-to for most ML algorithms besides neural networks)
- [StatQuest YouTube Channel](https://www.youtube.com/c/joshstarmer)
- [DeepLearningAI YouTube Channel](https://www.youtube.com/c/Deeplearningai/videos)
- [Towards Data Science](https://towardsdatascience.com/) (articles about data science and machine learning, some involving example blocks of code)
- Advance searching [arxiv](https://arxiv.org/search/advanced) (e.g. search term "machine learning" in Abstract for Subject astro-ph) to see what others are doing currently
- Google, YouTube, Wikipedia and ChatGPT (confirm results with external sources) in general
  - [K-means and Hierarchical Clustering Video](https://www.youtube.com/watch?v=QXOkPvFM6NU)
  - [GMM Video](https://www.youtube.com/watch?v=q71Niz856KE)

<a id="about"></a>
## About this Notebook

**Author:** Fred Dauphin, DeepWFC3

**Updated on:** 2024-10-31

<a id="cite"></a>
## Citations

If you use `numpy`, `matplotlib`, `sklearn`, `umap`, or `hdbscan` for published research, please cite the authors. Follow these links for more information about citing `numpy`, `matplotlib`, `sklearn`, `umap`, and `hdbscan`:

* [Citing `numpy`](https://numpy.org/doc/stable/license.html)
* [Citing `matplotlib`](https://matplotlib.org/stable/users/project/license.html#:~:text=Matplotlib%20only%20uses%20BSD%20compatible,are%20acceptable%20in%20matplotlib%20toolkits.)
* [Citing `sklearn`](https://scikit-learn.org/stable/about.html#citing-scikit-learn)
* [Citing `umap`](https://github.com/lmcinnes/umap/blob/master/LICENSE.txt)
* [Citing `hdbscan`](https://hdbscan.readthedocs.io/en/latest/faq.html#q-i-want-to-cite-this-software-in-my-journal-publication-how-do-i-do-that)

***
[Top of Page](#title)